In [66]:
import json
import os
import pandas as pd

In [139]:
def read_atributes(file):

    atributos = {}
    remove = ['nFLows']

    with open(file, 'r') as signature_file:
        signature = json.load(signature_file)
        cardinalidade = signature['metadata']['cardinality']
        for step in signature['signature']:
            for atribute in signature['signature'][step]:
                if atribute not in remove:
                    atributos.setdefault(atribute, [signature['signature'][step][atribute]['value_min'],signature['signature'][step][atribute]['value_max']])

    return atributos, cardinalidade

In [157]:
sig_path = '/home/amanda/Desktop/dos/'

In [158]:
files = [os.path.join(sig_path, f) for f in os.listdir(sig_path) if os.path.isfile(os.path.join(sig_path, f))]

In [165]:
def query(atributos, cardinalidade, tabela):
    contar = ['nPortSrc', 'nPortDst', 'nAddrSrc', 'nAddrDst', 'nProto']
    dic = {'srcport': 'L4_SRC_PORT',
            'nPortSrc': 'L4_SRC_PORT',
            'destport': 'L4_DST_PORT',
            'nPortDst': 'L4_DST_PORT',
            'nAddrSrc': 'IPV4_SRC_ADDR',
            'nAddrDst': 'IPV4_DST_ADDR',
            'proto': 'PROTOCOL',
            'nProto': 'PROTOCOL',
            'nBytesSrc': 'IN_BYTES',
            'nBytesDst': 'OUT_BYTES',
            'nPktsSrc': 'IN_PKTS',
            'nPktsDst': 'OUT_PKTS'
            }
    
    prefixo = 'select '
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name in contar:
            prefixo += 'count(distinct ' + dic[atrib_name] + '),'
    
    if prefixo == 'select ':
        prefixo += 'count(*),'
        
    sql = prefixo[:-1] + ' from ' + tabela + ' where '
    statement = ''
    cont = 0
    queries = []
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name not in contar:
            statement += '(' + dic[atrib_name] + ' between ' + str(atrib_value[0]) + ' AND ' + str(atrib_value[1]) + ')'
            if cont < len(atributos.items()) - 1:
                statement += ' AND '
        cont = cont + 1
    
    minimo = 1
    for interval in range(1000,51000,1000):
        if cardinalidade == '1-N':
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_SRC_ADDR;'
        elif cardinalidade == 'N-1':
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_DST_ADDR;'
        else:
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ');'
        minimo = interval
        queries.append(sql + statement + sufixo)
        print(sql + statement + sufixo)

    return queries

In [166]:
queries = []
for file in files:
    print('\n\nAssinatura: ' + str(file))
    atributos, cardinalidade = read_atributes(file)
    queries.append(query(atributos, cardinalidade, 'bruteforce_flows'))



Assinatura: /home/amanda/Desktop/dos/dos4.sig
select count(*) from bruteforce_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND (idx between 1 AND 1000) group by IPV4_DST_ADDR;
select count(*) from bruteforce_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND (idx between 1000 AND 2000) group by IPV4_DST_ADDR;
select count(*) from bruteforce_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND (idx between 2000 AND 3000) group by IPV4_DST_ADDR;
select count(*) from bruteforce_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND (idx between 3000 AND 4000) group by IPV4_DST_ADDR;
select count(*) from bruteforce_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND (idx between 4000 AND 5000) group by IPV4_DST_ADDR;
select count(*) from bruteforce_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND (idx between 5000 AND 6000) g

In [125]:
import MySQLdb
def consultar(queries):
    db = MySQLdb.connect("localhost","root","1nstalation","ntopng")
    cursor = db.cursor()
    for query in queries:
        try:
            cursor.execute(query)
            results = cursor.fetchall()
        except:
            print ("Error: unable to fecth data")
    db.close()

ModuleNotFoundError: No module named 'MySQLdb'

In [126]:
atributos

{'destport': 22, 'proto': 6}

In [92]:
minimo = 1
for interval in range(1000,51000,1000):
    print(minimo,interval)
    minimo = interval

1 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 40000
40000 41000
41000 42000
42000 43000
43000 44000
44000 45000
45000 46000
46000 47000
47000 48000
48000 49000
49000 50000
